In [0]:
full_path = '/Volumes/formula1/landing/landing_zone_formula1'
database_name = 'formula1'
database_name_bronze = 'formula1.bronze'

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS formula1;
CREATE SCHEMA IF NOT EXISTS formula1.landing;
USE CATALOG formula1;

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS formula1.landing.landing_zone_formula1;

In [0]:
display(dbutils.fs.ls(full_path))

In [0]:
df = (spark
    .read
    .option("header", True)
    .option("inferSchema", True)
    .csv(full_path + '/circuits.csv'))

display(df)

In [0]:
for file in dbutils.fs.ls(full_path):
    path = file[0]
    file_name = file[1].replace('.csv', '')
    print(path, '---', file_name)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS formula1.bronze;

In [0]:
from pyspark.sql.functions import current_timestamp

for file in dbutils.fs.ls(full_path):
    path = file[0]
    file_name = file[1].replace(".csv", "")
    print(path, "---", file_name)

    df = spark.read.option("header", True).option("inferSchema", True).csv(path)

    df = df.withColumn("bronze_ingestion", current_timestamp())

    df.write.format("delta").mode("overwrite").saveAsTable(
        f"{database_name_bronze}.{file_name}"
    )

In [0]:
%sql
DESCRIBE HISTORY formula1.bronze.circuits;

In [0]:
%sql
SELECT * FROM formula1.bronze.circuits;